Redes neurais de classificação geralmente têm uma camada final com uma função sigmoid (para 1 saída) ou softmax (para múltiplas classes). Essa função entrega uma probabilidade entre 0 e 1 para cada classe.

Peça a saída raw da última camada da rede, antes de tomar a decisão.
Essa saída é a probabilidade da classe positiva (euploide).
Multiplique por 100 para transformar em porcentagem.

In [2]:
import pandas as pd
import joblib

# === 1. Carrega a planilha com os dados dos embriões ===
df = pd.read_excel("PlanilhaNumerica.xlsx")

# === 2. Carrega o modelo e o scaler treinados ===
modelo = joblib.load("melhor_modelo_mlp2.pkl")
scaler = joblib.load("scaler_mlp2.pkl")

# === 3. Prepara os dados ===
X = df.drop(columns=["Ploidia"])  # Remove a resposta real

# Remove a coluna 'Padrão' se existir para evitar erro no scaler
if 'Padrão' in X.columns:
    X = X.drop(columns=['Padrão'])

# Aplica a transformação usando o scaler treinado
X_scaled = scaler.transform(X)

# === 4. Faz a predição da classe (0 ou 1) ===
classes_preditas = modelo.predict(X_scaled)

# === 5. Adiciona ao DataFrame original ===
df["Classe_Prevista"] = classes_preditas

# === 6. Salva em nova planilha ===
df.to_excel("Planilha_Com_Classificacao.xlsx", index=False)

print("✅ Classificações salvas em 'Planilha_Com_Classificacao.xlsx'")

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- Padrão


## Pode testar isso aqui com a planilha nova de dados 

## Etapa 1: Lógica da Porcentagem

Vamos usar as variáveis morfocinéticas e morfológicas da planilha. Para cada variável:

- Se o valor do embrião estiver **na faixa favorável**, ele ganha a **pontuação cheia proporcional ao peso** daquela variável.
- Se estiver **fora da faixa favorável**, ganha **zero** (ou um valor intermediário, se quisermos refinar depois com mais precisão ou uma função contínua).

A **soma ponderada dos acertos** (variáveis que estiverem dentro da faixa favorável) dará a **porcentagem estimada de euploidia**, variando de **0 a 100**.

## Etapa 1: Lógica da Porcentagem

Vamos usar as variáveis morfocinéticas e morfológicas da planilha. Para cada variável:

- Se o valor do embrião estiver **na faixa favorável**, ele ganha a **pontuação cheia proporcional ao peso** daquela variável.
- Se estiver **fora da faixa favorável**, ganha **zero** (ou um valor intermediário, se quiser refinar depois com mais precisão ou uma função contínua).

A **soma ponderada dos acertos** (variáveis que estiverem dentro da faixa favorável) dará a **porcentagem estimada de euploidia**, variando de **0 a 100**.

---

## Etapa 2: Pesos Normalizados (baseados na correlação de Spearman ou evidência)

Aqui estão os pesos que usaremos, totalizando 100%:

| Variável     | Peso (%) |
|--------------|----------|
| Idade        | 28.25    |
| t5           | 13.56    |
| tB - tSB     | 15.82    |
| cc3          | 15.82    |
| Estágio      | 10.00    |
| KIDScore     | 10.00    |
| Morfologia   | 10.00    |
| s2           | 5.00     |
| cc2          | 5.00     |
| t5 - t2      | 5.00     |
| **Total**    | **100**  |

# Teste de classificação de ploidia

In [4]:
import pandas as pd
import joblib
import numpy as np

# 1. Carrega os dados
df = pd.read_excel("PlanilhaNumerica.xlsx")

# 2. Carrega o modelo e o scaler treinados
modelo = joblib.load("melhor_modelo_mlp_20250610_113659.pkl")
scaler = joblib.load("scaler_mlp_20250610_113659.pkl")

# 3. Prepara os dados
X = df.drop(columns=["Ploidia"], errors="ignore")

# 4. Preenche as colunas faltantes com 0 (ex: 'Padrão')
colunas_treinadas = scaler.feature_names_in_
for col in colunas_treinadas:
    if col not in X.columns:
        X[col] = 0  # Ou use np.nan e depois fillna com média, se preferir

# 5. Garante a ordem correta das colunas
X = X[colunas_treinadas]

# 6. Aplica o scaler
X_scaled = scaler.transform(X)

# 7. Faz a predição
classes_preditas = modelo.predict(X_scaled)

# 8. Adiciona ao DataFrame original
df["Classe_Prevista"] = classes_preditas

# 9. Salva planilha com todas as classificações
df.to_excel("Planilha_Com_Classificacao.xlsx", index=False)

# 10. Salva planilha apenas com os embriões euploides
df_euploides = df[df["Classe_Prevista"] == 1]
df_euploides.to_excel("Planilha_Somente_Euploidia.xlsx", index=False)

print("✅ Concluído com sucesso!")
print("📄 Todas as classes: Planilha_Com_Classificacao.xlsx")
print("📄 Somente euploides: Planilha_Somente_Euploidia.xlsx")

✅ Concluído com sucesso!
📄 Todas as classes: Planilha_Com_Classificacao.xlsx
📄 Somente euploides: Planilha_Somente_Euploidia.xlsx


In [3]:
import joblib
import pandas as pd
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import train_test_split

# === CARREGAR OS DADOS ===
df_original = pd.read_excel("PlanilhaNumerica.xlsx")
X = df_original.drop(columns=["Ploidia"], errors="ignore")
y = df_original["Ploidia"]

# === CARREGAR MODELO E SCALER SALVOS ===
modelo = joblib.load("melhor_modelo_mlp_20250610_113659.pkl")
scaler = joblib.load("scaler_mlp_20250610_113659.pkl")

# === GARANTIR QUE AS COLUNAS ESTÃO COMPLETAS ===
colunas_treinadas = scaler.feature_names_in_

for col in colunas_treinadas:
    if col not in X.columns:
        X[col] = 0  # ou np.nan e depois preencher com média, se preferir

# === ORDENAR AS COLUNAS NA ORDEM CERTA ===
X = X[colunas_treinadas]

# === DIVIDIR OS DADOS ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# === ESCALAR ===
X_test_scaled = scaler.transform(X_test)

# === FAZER PREVISÕES ===
y_pred = modelo.predict(X_test_scaled)
y_proba = modelo.predict_proba(X_test_scaled)[:, 1]

# === MÉTRICAS ===
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()

accuracy = (tp + tn) / (tp + tn + fp + fn)
recall_1 = tp / (tp + fn) if (tp + fn) > 0 else 0
recall_0 = tn / (tn + fp) if (tn + fp) > 0 else 0
auc = roc_auc_score(y_test, y_proba)

# === EXIBIR RESULTADOS ===
print(f"Acurácia (accuracy): {accuracy:.3f}")
print(f"Recall para Euploide (recall_1): {recall_1:.3f}")
print(f"Recall para Aneuploide (recall_0): {recall_0:.3f}")
print(f"AUC: {auc:.3f}")

Acurácia (accuracy): 0.882
Recall para Euploide (recall_1): 0.750
Recall para Aneuploide (recall_0): 1.000
AUC: 0.944


## parei em:
- fazer ele classificar uma planilha, e retornar uma nova planilha somente com os euploides.
- depois fazer a porcentagem de euploidia somente com essa